<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
classification_nlp_tfidf_pca_xgboost_gender
</font>
</h1>

In [1]:
import numpy as np
import pandas as pd 
from hazm import stopwords_list, Normalizer, word_tokenize, Lemmatizer
import re
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|gender|جنسیت (ستون هدف)|
|age|بازه سنی کاربر|
|fullname|نامی که در پروفایل شبکه اجتماعی نوشته شده است |
|username|نام کاربری|
|biography|بیوگرافی شبکه اجتماعی کاربر|
|follower_count|تعداد دنبال‌کننده‌های کاربر|
|following_count|تعداد افرادی که کاربر را دنبال می‌کنند|
|is_business|کسب‌وکاری بودن حساب کاربری|
|is_verified|تایید شده بودن حساب کاربری|
|is_private|خصوصی بودن حساب کاربری|
    
</font>
</div>
</center>



<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    
|سن واقعی کاربران|عدد نگاشت شده|
|:------:|:---:|
|1|کمتر از ۱۸ سال|
|2|از ۱۹ تا ۲۹ سال|
|3|از ۳۰ تا ۴۰ سال|
|4|بالاتر از ۴۰ سال|
    
</font>
</div>
</center>


In [2]:
train = pd.read_csv('../data/train_data.csv')
test = pd.read_csv('../data/test_data.csv')
train

,gender,age,fullname,username,biography,follower_count,following_count,is_business,is_verified,is_private
0,man,2,Farshid,mr_gh_farshid,دردا ک در این بادیه بسیار دویدیم...\nGlory man...,1604.0,1407.0,0.0,0.0,0.0
1,woman,2,zahr@72,zahra.roozbahani72,"خواهی که زکوچ در امان برگردی\nباید که به جان ,...",67.0,501.0,0.0,0.0,0.0
2,woman,2,ms farahnaz♥,___lady.farahnazi.__,"Having you, is all I wish for \nداشتنت، تمامِ...",0.0,0.0,0.0,0.0,0.0
3,woman,1,Lena.mommy farzan,mommy.lena3361,دردونه من لنا کوچولو,0.0,0.0,0.0,0.0,0.0
4,woman,2,Narsis Asadollahi,_l.aurora.l_,I am an animation student\n🎧🎼🎨⚓️🤍 \n@general.m...,200.0,328.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7995,woman,2,Ŋεgɨŋ,negiiin_bahrmandi,﷽\nAllah IS Enough FoR Me?\n♥️?,0.0,0.0,0.0,0.0,0.0
7996,man,3,h🗯abdi🗯offìcial,h.abdi.official,حقوقی,0.0,0.0,1.0,0.0,0.0
7997,woman,2,⚜رویا احمدی⚜,roya.ahmadi.k,مهندس صنایع👩‍🔧🏭 Industrial engineer\nمعمار👩‍💻👷...,1260.0,1167.0,0.0,0.0,0.0
7998,man,3,لرستان &خرم اباد,erfanpouersif,khoramabad,0.0,0.0,0.0,0.0,0.0


In [3]:
label = train['gender']
label = label.map({'man':0, 'woman':1})
train.drop(columns=['gender'], inplace=True)

In [4]:
train.drop(columns=['fullname', 'follower_count','following_count','is_business'], inplace=True)

test.drop(columns=['fullname','follower_count','following_count','is_business'], inplace=True)

In [5]:
train.head(3)

,age,username,biography,is_verified,is_private
0,2,mr_gh_farshid,دردا ک در این بادیه بسیار دویدیم...\nGlory man...,0.0,0.0
1,2,zahra.roozbahani72,"خواهی که زکوچ در امان برگردی\nباید که به جان ,...",0.0,0.0
2,2,___lady.farahnazi.__,"Having you, is all I wish for \nداشتنت، تمامِ...",0.0,0.0


In [6]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()
stopwords = set(stopwords_list())
pattern_html = re.compile(r"<.*?>", flags=re.DOTALL)
pattern_num = re.compile(r"\d+", flags=re.UNICODE)
pattern_punc = re.compile(r"[^\w\s]")
pattern_underscore = re.compile(r"_+") 

def preprocessing(text):
    if text is None:
        return ""
    try:
        text = normalizer.normalize(text)
    except Exception:
        pass

    text = pattern_html.sub(" ", text)
    text = pattern_underscore.sub(" ", text)
    text = pattern_num.sub(" ", text)
    text = pattern_punc.sub(" ", text)
    
    try:
        text = text.lower()
    except Exception:
        pass
    
    tokens = word_tokenize(text)

    filtered = ''
    for w in tokens:
        if w and w not in stopwords:
            w2 = lemmatizer.lemmatize(w)
            if "#" in w2:
                w2 = w2.split("#")[0]
            filtered = filtered + ' ' + w2
    return filtered.strip()



In [7]:
train['biography'] = train['biography'].apply( lambda r : preprocessing(r))
train['username'] = train['username'].apply( lambda r : preprocessing(r))

test['biography'] = test['biography'].apply( lambda r : preprocessing(r))
test['username'] = test['username'].apply( lambda r : preprocessing(r))

In [8]:
train.head(2)

,age,username,biography,is_verified,is_private
0,2,mr gh farshid,دردا ک بادیه دوید glory man u dey rip paulnewman,0.0,0.0
1,2,zahra roozbahani,خواهی_که زکوچ امان برگشت جان زجان جان برگشت,0.0,0.0


In [9]:
corpus = train.apply(lambda r: r['biography'] + " " + r['username'], axis=1)
test_corpus = test.apply(lambda r: r['biography'] + " " + r['username'], axis=1)
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(corpus)
train_final_counts = tfidf_vect.transform(corpus)
test_final_counts = tfidf_vect.transform(test_corpus)

print(tfidf_vect.get_feature_names_out())
print(train_final_counts.toarray()[0])
print(train_final_counts.toarray()[1])

['aa' 'aaa' 'aaaa' ... '𝚒𝚗𝚜𝚝𝚛𝚞𝚌𝚝𝚘𝚛' '𝚘𝚏' '𝚝𝚘']
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]


In [10]:
feature_names = tfidf_vect.get_feature_names_out()
train_TF_IDF = pd.DataFrame(train_final_counts.toarray(), columns=feature_names)
train_TF_IDF.head(2)

,aa,aaa,aaaa,aaaaaaaa,aaaaaeuaisooqm,aaaysaaan,aabbasi,aad,aafsane,aaftab,...,𝚋𝚒𝚕𝚕𝚒𝚘𝚗𝚜,𝚋𝚘𝚕𝚍,𝚌𝚘𝚖𝚎,𝚍𝚛𝚎𝚊𝚖,𝚐𝚘,𝚑𝚎𝚊𝚛𝚝𝚜,𝚒𝚗,𝚒𝚗𝚜𝚝𝚛𝚞𝚌𝚝𝚘𝚛,𝚘𝚏,𝚝𝚘
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
test_TF_IDF = pd.DataFrame(test_final_counts.toarray(), columns=feature_names)

In [12]:
train_TF_IDF.shape

(8000, 24497)

In [13]:
pca = PCA(n_components=700)
train_reduced = pca.fit_transform(train_TF_IDF)
test_reduced = pca.transform(test_TF_IDF)

In [14]:
train_reduced.shape

(8000, 700)

In [15]:
selected_cols = train[['age', 'is_verified', 'is_private']].reset_index(drop=True)
train_reduced = pd.DataFrame(train_reduced)
train_reduced = train_reduced.reset_index(drop=True)

final_train = pd.concat([selected_cols, train_reduced], axis=1)


In [16]:
train.head()

,age,username,biography,is_verified,is_private
0,2,mr gh farshid,دردا ک بادیه دوید glory man u dey rip paulnewman,0.0,0.0
1,2,zahra roozbahani,خواهی_که زکوچ امان برگشت جان زجان جان برگشت,0.0,0.0
2,2,lady farahnazi,having you is all i wish for داشتن آرزو ست داش...,0.0,0.0
3,1,mommy lena,دردونه لنا کوچولو,0.0,0.0
4,2,l aurora l,i am an animation student general motion,0.0,0.0


In [17]:
selected_cols = test[['age', 'is_verified', 'is_private']].reset_index(drop=True)
test_reduced = pd.DataFrame(test_reduced)
test_reduced = test_reduced.reset_index(drop=True)

final_test = pd.concat([selected_cols, test_reduced], axis=1)


In [18]:
final_train.shape

(8000, 703)

In [19]:
X_train, X_val, y_train, y_val = train_test_split(final_train, label, random_state=42, test_size=0.20)

In [20]:
model = XGBClassifier(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    eval_metric='logloss' 
)

model.fit(X_train, y_train)



,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.9
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [21]:
from sklearn.metrics import f1_score

pred = model.predict(X_val)
print("F1:", f1_score(y_val, pred))


F1: 0.7600254614894971


In [22]:
pred_test = model.predict(final_test)

pred_test = pd.Series(pred_test)

pred_test = pred_test.map({0: 'man', 1: 'woman'})

submission = pd.DataFrame({'gender': pred_test})

submission

,gender
0,woman
1,man
2,woman
3,woman
4,woman
...,...
1995,woman
1996,woman
1997,man
1998,woman
